In [51]:
import geopandas as gpd
import pandas as pd

In [52]:
victoria_gdf = gpd.read_file('../data/Statistical_area_level2/victoria.geojson')

In [53]:
victoria_gdf

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78282 -37.56666, 143.75558 -37.5..."
1,201011002,Ballarat,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,12.3787,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.81896 -37.55582, 143.81644 -37.5..."
2,201011005,Buninyong,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,51.5855,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.84171 -37.61596, 143.84176 -37.6..."
3,201011006,Delacombe,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,34.1607,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.75050 -37.59119, 143.75044 -37.5..."
4,201011007,Smythes Creek,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,104.7274,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.73296 -37.62333, 143.73263 -37.6..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,217041478,Moyne - West,0,No change,21704,Warrnambool,217,Warrnambool and South West,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,2037.7626,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((142.00870 -38.41715, 142.00876..."
520,217041479,Warrnambool - North,0,No change,21704,Warrnambool,217,Warrnambool and South West,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,62.4199,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((142.43668 -38.35544, 142.43658 -38.3..."
521,217041480,Warrnambool - South,0,No change,21704,Warrnambool,217,Warrnambool and South West,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,120.1957,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((142.45281 -38.39126, 142.45230 -38.3..."
522,297979799,Migratory - Offshore - Shipping (Vic.),0,No change,29797,Migratory - Offshore - Shipping (Vic.),297,Migratory - Offshore - Shipping (Vic.),29799,Migratory - Offshore - Shipping (Vic.),2,Victoria,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None


In [54]:
school_df = pd.read_csv("../data/external_SA2/complete_school_location.csv", encoding='latin-1')

In [55]:
school_df['coordinates'] = school_df.apply(lambda row: [row['Y'], row['X']], axis=1)

In [56]:
school_df['coordinates']

0        [-38.084432, 145.3643874]
1        [28.8155588, -96.9814539]
2        [-37.5611479, 143.858787]
3        [-37.8103129, 144.972154]
4       [-37.8768613, 145.1490967]
                   ...            
2252        [-37.64875, 145.08148]
2253         [-37.53046, 144.9052]
2254        [-37.97324, 145.31589]
2255        [-37.88319, 145.29327]
2256        [-37.62974, 144.88566]
Name: coordinates, Length: 2257, dtype: object

In [57]:
import geopandas as gpd
from shapely.geometry import Point

# Your property data
school_metadata = school_df

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
#school_metadata['coordinates'] = school_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in school_metadata['coordinates']]
names = school_metadata['School_Type'].tolist()

# Convert to GeoDataFrame
school_gdf = gpd.GeoDataFrame({'name': names, 'geometry': coords})

In [58]:
school_gdf

,name,geometry
0,college,POINT (145.36439 -38.08443)
1,college,POINT (-96.98145 28.81556)
2,college,POINT (143.85879 -37.56115)
3,university,POINT (144.97215 -37.81031)
4,college,POINT (145.14910 -37.87686)
...,...,...
2252,Secondary,POINT (145.08148 -37.64875)
2253,Primary,POINT (144.90520 -37.53046)
2254,Primary,POINT (145.31589 -37.97324)
2255,Secondary,POINT (145.29327 -37.88319)


In [59]:
joined_gdf = gpd.sjoin(school_gdf, victoria_gdf, how="inner", op="within")

/Users/ivysu/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/38/85yvxj112kj5cp41m83y71800000gn/T/ipykernel_48563/3230858767.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined_gdf = gpd.sjoin(school_gdf, victoria_gdf, how="inner", op="within")


In [60]:
school_SA2 = joined_gdf[['name', 'geometry', 'SA2_CODE21', 'SA2_NAME21']]

In [61]:
school_SA2.rename(columns={'name':'School_Type', 'geometry': 'School_Geometry', 'SA2_CODE21	': 'School_SA2_Code', 'SA2_NAME21': 'School_SA2_Name'})


,School_Type,School_Geometry,SA2_CODE21,School_SA2_Name
0,college,POINT (145.36439 -38.08443),212031555,Clyde North - North
1082,Primary,POINT (145.37283 -38.08273),212031555,Clyde North - North
2062,Pri/Sec,POINT (145.36380 -38.08468),212031555,Clyde North - North
2,college,POINT (143.85879 -37.56115),201011002,Ballarat
82,Primary,POINT (143.85394 -37.56401),201011002,Ballarat
...,...,...,...,...
2065,Pri/Sec,POINT (145.46732 -38.06190),212011550,Pakenham - North West
2213,Primary,POINT (145.43271 -38.06277),212011550,Pakenham - North West
2088,Pri/Sec,POINT (144.56028 -37.70028),213041571,Brookfield
2184,Pri/Sec,POINT (144.33707 -38.23046),203021485,Charlemont


In [62]:
school_SA2.to_csv('../data/raw/school_SA2.csv', index=False) 